# Resumen
En este colab se llevara a cabo el proceso para la descarga de imagenes del dataset de airbnb. Utilizaremos el campo "Thumbnail Url" que proporciona imagenes de una tamaño adecuado a nuestras necesidades (216x144).

Tambien se generara un dataset nuevo (airbnb-images-clean.csv) con aquellos registros que tengan imagenes asociadas con el tamaño indicado arriba, borrandose el resto.

Este dataset contendra un nuevo campo llamado image_path que contendra el el path relativo de cada imagen descargada. 

El nombre de las imagenes estara asociado al campo ID de cada registro  

# Montar directorio Drive en colab
En primer lugar vamos a montar el directorio de Drive para poder utilizar los ficheros de carga y para guardar de manera permanente otros ficheros de interes

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Definimos una serie de constantes para establecer el path del proyecto dentro de nuestro Google Drive asi como el directorio donde se almacenaran las imagenes

In [0]:
from os.path import join
ROOT = "/content/gdrive"
PROJ = "My Drive/Public/Practica_Deep_Learning_DA" 
PROJECT_PATH = join(ROOT, PROJ)

images_dir = 'images'
IMAGES_PATH = join(PROJECT_PATH, images_dir)

In [0]:
#Coprobamos que nuestro Google Drive esta corectamente montado
!ls "/content/gdrive/My Drive/Public/Practica_Deep_Learning_DA"

data			pictures	   README.md
descarga_imagenes.ipyn	practica_dl.ipynb  regresion_ml_data.ipynb


# Librerías y funciones

## Definimos los Imports

In [0]:
import numpy as np
import pandas as pd
import urllib.request


## Función para la descarga de Imagenes

In [0]:
def download_image(id,url,file_path):    
  try:
    filename = 'image-{}.jpg'.format(id)
    #full_path = '{}{}'.format(file_path, filename)
    full_path = join(file_path, filename)
    urllib.request.urlretrieve(url, full_path)
    return full_path
  except:
    return 404


# Proceso de Descarga de Imagenes y generacion del nuevo Dataset 

## Download de Imagenes y carga en el Dataset


In [0]:
#Cargamos el fichero inicial
full_airbnb = pd.read_csv(join(PROJECT_PATH,'data/airbnb-listings.csv'),sep=';', decimal='.')

#Incluimos una nueva columna con NaN donde ira el path de las imagenes descargadas
full_airbnb["image_path"] = np.nan

#Tratamiento de las url y descarga de imagenes
from time import time
#Vamos a poner tres contadores para ver cuantas imagenes han ido bien, cuantas no tenian url (NaN) y cuantas han dado error 404
#Tambien incluimos uno global
cont_images_ok = 0
cont_urls_nan = 0
cont_urls_404 = 0
cont_global = 0

#Creamos 3 listas donde almacenaremos los IDs para cada caso
ids_image_ok = []
ids_url_nan = []
ids_url_404 = []

#Vamos a visualizar cuanto tarda el proceso
start_time = time()

for index, row in full_airbnb.iterrows():
  cont_global +=1
  if cont_global%100 == 0:
    print('LLevamos {} Registros tratados'.format(cont_global))
  if type(row['Thumbnail Url'])==str:
    id = row['ID']
    url = row['Thumbnail Url']
    image_path = download_image(id,url,IMAGES_PATH)
    if image_path == 404:
      cont_urls_404 +=1
      ids_url_404.append(id)
    else:
      cont_images_ok +=1
      ids_image_ok.append(id)
      full_airbnb.loc[index,"image_path"] = image_path
  else:
    cont_urls_nan +=1
    ids_url_nan.append(id)

elapsed_time = time() - start_time
print("Elapsed time: %0.10f seconds." % elapsed_time)
print('Total Registros tratados: {}'.format(cont_global))
print('Se han subido {} imagenes correctamente'.format(cont_images_ok))
print('Registros sin URL(NaN): {}'.format(cont_urls_nan))
print('Registros con error 404: {}'.format(cont_urls_404))

LLevamos 100 Registros tratados
LLevamos 200 Registros tratados
LLevamos 300 Registros tratados
LLevamos 400 Registros tratados
LLevamos 500 Registros tratados
LLevamos 600 Registros tratados
LLevamos 700 Registros tratados
LLevamos 800 Registros tratados
LLevamos 900 Registros tratados
LLevamos 1000 Registros tratados
LLevamos 1100 Registros tratados
LLevamos 1200 Registros tratados
LLevamos 1300 Registros tratados
LLevamos 1400 Registros tratados
LLevamos 1500 Registros tratados
LLevamos 1600 Registros tratados
LLevamos 1700 Registros tratados
LLevamos 1800 Registros tratados
LLevamos 1900 Registros tratados
LLevamos 2000 Registros tratados
LLevamos 2100 Registros tratados
LLevamos 2200 Registros tratados
LLevamos 2300 Registros tratados
LLevamos 2400 Registros tratados
LLevamos 2500 Registros tratados
LLevamos 2600 Registros tratados
LLevamos 2700 Registros tratados
LLevamos 2800 Registros tratados
LLevamos 2900 Registros tratados
LLevamos 3000 Registros tratados
LLevamos 3100 Regis

## Eliminacion de los registros sin imagen asociada

Ya sea porque no tienen URL en el campo "Thumbnail" o dicha URL no es valida (Error 404).

Tambien se creara un nuevo dataset llamado airbnb-images.csv 

In [0]:
full_airbnb_images = full_airbnb.drop(full_airbnb[full_airbnb['image_path'].isnull()].index)
#Guardamos este dataframe como un cvs
full_airbnb_images.to_csv(join(PROJECT_PATH,'data/airbnb-images.csv'), sep=';', decimal='.', index=False)

## Eliminación de registros con imagenes que no se corresponden con el tamaño establecido.

Se elimininaran aquellos registros cuya imagen asociada no se correposdan con el tamaño de 216x144. 

En nuestro caso solo son 6 y son demasiado pequeñas para plantearnos ajustarlas al tamaño deseado. Si fueran más o tuvieran otros tamaños nos podriamos plantear llevar a cabo un resize de las mismas.

In [0]:
#Por ultimo comprobamos si hay alguna imagen con un tamaño que no se adapta al tamaño establecido (216x144) para eliminarla tambien
from PIL import Image
from time import time

def get_num_pixels(filepath):
  width, height = Image.open(filepath).size
  return (width,height)
# Creamos una lista para guardar las imagenes que no tiene el tamaño standar de 216x144
image_other_size = []

cont_global=0
start_time = time()
#Asi items es un vector con el index
#for items in full_airbnb['image_path'].iteritems():
#Asi items solo contiene el valor
for items in full_airbnb_images['image_path']:
  cont_global +=1
  if cont_global%100 == 0:
    print('LLevamos {} Registros tratados'.format(cont_global))
  if not pd.isnull(items):
    image_size = get_num_pixels(items)
    if image_size != (216, 144):
      image_other_size.append(items)

elapsed_time = time() - start_time
print("Elapsed time: %0.10f seconds." % elapsed_time)
print('Imagenes con tamaño distinto al establecido (216x144): {}'.format(len(image_other_size)))



In [0]:
#Eliminamos los registros de esas imagenes
full_airbnb_images_clean = full_airbnb_images.drop(full_airbnb_images[full_airbnb_images['image_path'].isin(image_other_size)].index)


In [0]:
full_airbnb_images_clean.shape

(11936, 90)

In [0]:
#Una vez limpiado el dataset con las imagenes que vamos a tratar lo guardamos en un fichero.
full_airbnb_images_clean.to_csv(join(PROJECT_PATH,'data/airbnb-images-clean.csv'), sep=';', decimal='.', index=False)


# Carga del Fichero con imagenes
Cargamos el dataset que ya contiene las imagenes limpias

In [0]:
full_airbnb_images = pd.read_csv(join(PROJECT_PATH,'data/airbnb-images-clean.csv'),sep=';', decimal='.')


In [0]:
len(full_airbnb_images)

14780

In [0]:
type(full_airbnb_images[~pd.isnull(full_airbnb_images['image_path'])]['image_path'].values)

pandas.core.series.Series

In [0]:
type(full_airbnb_images['image_path'].values)

numpy.ndarray

Ahora vamos a crear los tensores que representaran las distintas imagenes y las guardaremos en un nuevo campo en el dataframe

In [0]:
import cv2
array_images = full_airbnb_images['image_path'].values
N = len(array_images)
data = np.empty((N, 3, 144, 216), dtype=np.uint8)
for i, fpath in enumerate(array_images):
  img = cv2.imread(fpath, cv2.IMREAD_COLOR)
  data[i, ...] = img.transpose(2, 0, 1)